# Train 
The network can be trained using three interfaces:

- python, via `das.train.train`
- the command-line interface `das train`.
- the GUI - see the [GUI tutorial](/tutorials_gui/train)

Training will:

- load train/val/test data form a dataset
- initialize the network
- save all parameters for reproducibility
- train the network and save the best network to disk
- run inference and evaluate the network using the test data.

The names of files created during training start with an optional prefix and the time stamp of the start time of training, as in `my-awesome-prefix_20192310_091032`. Typically, three files are created:
- `*_params.yaml` - training parameters etc.
- `*_model.h5` -  model architecture and weights
- `*_results.h5` - predictions and evaluation results for the test set (only created if the training dataset contains a test set)

## Training using python
Training is done using the `train` function in the `das.train` module:

In [1]:
import das.train
help(das.train.train)

Help on function train in module das.train:

train(*, data_dir: str, y_suffix: str = '', save_dir: str = './', save_prefix: str = None, model_name: str = 'tcn', nb_filters: int = 16, kernel_size: int = 16, nb_conv: int = 3, use_separable: List[bool] = False, nb_hist: int = 1024, ignore_boundaries: bool = True, batch_norm: bool = True, nb_pre_conv: int = 0, pre_kernel_size: int = 3, pre_nb_filters: int = 16, pre_nb_conv: int = 2, verbose: int = 2, batch_size: int = 32, nb_epoch: int = 400, learning_rate: float = None, reduce_lr: bool = False, reduce_lr_patience: int = 5, fraction_data: float = None, seed: int = None, batch_level_subsampling: bool = False, tensorboard: bool = False, log_messages: bool = False, nb_stacks: int = 2, with_y_hist: bool = True, x_suffix: str = '')
    Train a DAS network.
    
    Args:
        data_dir (str): Path to the directory or file with the dataset for training.
                        Accepts npy-dirs (recommended), h5 files or zarr files.
           

The output of the `train` function is fairly verbose, to help with troubleshooting:

- run time parameters
- information on the size of the training and validation data
- network architecture
- training progress (training and validation loss)
- after training, a classification report for the test data (if test data exist in the dataset)

To demonstrate the outputs of `train`, the following trains a small network on a small dataset to annotate pulse and sine song from _Drosophila melanogater_. Expected performance (f1-score) is about 75%.

In [2]:
das.train.train(model_name='tcn',  # see `das.models` for valid model_names
                data_dir='tutorial_dataset.npy', 
                save_dir='res',
                nb_hist=256,
                kernel_size=16,
                nb_filters=16,
                ignore_boundaries=True,
                verbose=1,
                nb_epoch=4,
                log_messages=True)

INFO:root:loading data
INFO:root:Parameters:
INFO:root:{'data_dir': 'tutorial_dataset.npy', 'y_suffix': '', 'save_dir': 'res', 'save_prefix': '', 'model_name': 'tcn', 'nb_filters': 16, 'kernel_size': 16, 'nb_conv': 3, 'use_separable': False, 'nb_hist': 256, 'ignore_boundaries': True, 'batch_norm': True, 'nb_pre_conv': 0, 'pre_kernel_size': 3, 'pre_nb_filters': 16, 'pre_nb_conv': 2, 'verbose': 1, 'batch_size': 32, 'nb_epoch': 4, 'reduce_lr': False, 'reduce_lr_patience': 5, 'fraction_data': None, 'seed': None, 'batch_level_subsampling': False, 'tensorboard': False, 'log_messages': True, 'nb_stacks': 2, 'with_y_hist': True, 'x_suffix': '', 'sample_weight_mode': 'temporal', 'data_padding': 48, 'return_sequences': True, 'stride': 160, 'y_offset': 0, 'output_stride': 1, 'class_names': ['noise', 'pulse', 'sine'], 'class_names_pulse': ['noise', 'pulse'], 'class_names_sine': ['noise', 'sine'], 'class_types': ['segment', 'event', 'segment'], 'class_types_pulse': ['segment', 'event'], 'class_type

```
INFO:root:loading data
INFO:root:Parameters:
INFO:root:{'data_dir': 'tutorial_dataset.npy', 'y_suffix': '', 'save_dir': 'res', 'save_prefix': '', 'model_name': 'tcn', 'nb_filters': 16, 'kernel_size': 16, 'nb_conv': 3, 'use_separable': False, 'nb_hist': 256, 'ignore_boundaries': True, 'batch_norm': True, 'nb_pre_conv': 0, 'pre_kernel_size': 3, 'pre_nb_filters': 16, 'pre_nb_conv': 2, 'verbose': 1, 'batch_size': 32, 'nb_epoch': 4, 'reduce_lr': False, 'reduce_lr_patience': 5, 'fraction_data': None, 'seed': None, 'batch_level_subsampling': False, 'tensorboard': False, 'log_messages': True, 'nb_stacks': 2, 'with_y_hist': True, 'x_suffix': '', 'sample_weight_mode': 'temporal', 'data_padding': 48, 'return_sequences': True, 'stride': 160, 'y_offset': 0, 'output_stride': 1, 'class_names': ['noise', 'pulse', 'sine'], 'class_names_pulse': ['noise', 'pulse'], 'class_names_sine': ['noise', 'sine'], 'class_types': ['segment', 'event', 'segment'], 'class_types_pulse': ['segment', 'event'], 'class_types_sine': ['segment', 'segment'], 'filename_endsample_test': [], 'filename_endsample_train': [], 'filename_endsample_val': [], 'filename_startsample_test': [], 'filename_startsample_train': [], 'filename_startsample_val': [], 'filename_train': [], 'filename_val': [], 'samplerate_x_Hz': 10000, 'samplerate_y_Hz': 10000, 'filename_test': [], 'nb_freq': 1, 'nb_channels': 1, 'nb_classes': 3, 'first_sample_train': 0, 'last_sample_train': None, 'first_sample_val': 0, 'last_sample_val': None}
INFO:root:preparing data
INFO:root:Training data:
INFO:root:AudioSequence with 3992 batches each with 32 items.
   Total of 20440005 samples with
   each x=(1,) and
   each y=(3,)
INFO:root:Validation data:
INFO:root:AudioSequence with 812 batches each with 32 items.
   Total of 4160001 samples with
   each x=(1,) and
   each y=(3,)
INFO:root:building network
WARNING:tensorflow:From /Users/clemens10/miniconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1635: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
WARNING:tensorflow:From /Users/clemens10/miniconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1635: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:root:None
INFO:root:start training
Model: "TCN"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_1 (InputLayer)            [(None, 256, 1)]     0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 256, 16)      32          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 256, 16)      4112        conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 16)      0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 256, 16)      0           activation[0][0]                 
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 256, 16)      0           lambda[0][0]                     
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 256, 16)      272         spatial_dropout1d[0][0]          
__________________________________________________________________________________________________
add (Add)                       (None, 256, 16)      0           conv1d[0][0]                     
                                                                 conv1d_2[0][0]                   
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 256, 16)      4112        add[0][0]                        
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 256, 16)      0           conv1d_3[0][0]                   
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 256, 16)      0           activation_1[0][0]               
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 256, 16)      0           lambda_1[0][0]                   
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 256, 16)      272         spatial_dropout1d_1[0][0]        
__________________________________________________________________________________________________
add_1 (Add)                     (None, 256, 16)      0           add[0][0]                        
                                                                 conv1d_4[0][0]                   
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 256, 16)      4112        add_1[0][0]                      
__________________________________________________________________________________________________
activation_2 (Activation)       (None, 256, 16)      0           conv1d_5[0][0]                   
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 256, 16)      0           activation_2[0][0]               
__________________________________________________________________________________________________
spatial_dropout1d_2 (SpatialDro (None, 256, 16)      0           lambda_2[0][0]                   
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 256, 16)      272         spatial_dropout1d_2[0][0]        
__________________________________________________________________________________________________
add_2 (Add)                     (None, 256, 16)      0           add_1[0][0]                      
                                                                 conv1d_6[0][0]                   
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 256, 16)      4112        add_2[0][0]                      
__________________________________________________________________________________________________
activation_3 (Activation)       (None, 256, 16)      0           conv1d_7[0][0]                   
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 256, 16)      0           activation_3[0][0]               
__________________________________________________________________________________________________
spatial_dropout1d_3 (SpatialDro (None, 256, 16)      0           lambda_3[0][0]                   
__________________________________________________________________________________________________
conv1d_8 (Conv1D)               (None, 256, 16)      272         spatial_dropout1d_3[0][0]        
__________________________________________________________________________________________________
add_3 (Add)                     (None, 256, 16)      0           add_2[0][0]                      
                                                                 conv1d_8[0][0]                   
__________________________________________________________________________________________________
conv1d_9 (Conv1D)               (None, 256, 16)      4112        add_3[0][0]                      
__________________________________________________________________________________________________
activation_4 (Activation)       (None, 256, 16)      0           conv1d_9[0][0]                   
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 256, 16)      0           activation_4[0][0]               
__________________________________________________________________________________________________
spatial_dropout1d_4 (SpatialDro (None, 256, 16)      0           lambda_4[0][0]                   
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 256, 16)      272         spatial_dropout1d_4[0][0]        
__________________________________________________________________________________________________
add_4 (Add)                     (None, 256, 16)      0           add_3[0][0]                      
                                                                 conv1d_10[0][0]                  
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 256, 16)      4112        add_4[0][0]                      
__________________________________________________________________________________________________
activation_5 (Activation)       (None, 256, 16)      0           conv1d_11[0][0]                  
__________________________________________________________________________________________________
lambda_5 (Lambda)               (None, 256, 16)      0           activation_5[0][0]               
__________________________________________________________________________________________________
spatial_dropout1d_5 (SpatialDro (None, 256, 16)      0           lambda_5[0][0]                   
__________________________________________________________________________________________________
conv1d_12 (Conv1D)              (None, 256, 16)      272         spatial_dropout1d_5[0][0]        
__________________________________________________________________________________________________
add_5 (Add)                     (None, 256, 16)      0           add_4[0][0]                      
                                                                 conv1d_12[0][0]                  
__________________________________________________________________________________________________
conv1d_13 (Conv1D)              (None, 256, 16)      4112        add_5[0][0]                      
__________________________________________________________________________________________________
activation_6 (Activation)       (None, 256, 16)      0           conv1d_13[0][0]                  
__________________________________________________________________________________________________
lambda_6 (Lambda)               (None, 256, 16)      0           activation_6[0][0]               
__________________________________________________________________________________________________
spatial_dropout1d_6 (SpatialDro (None, 256, 16)      0           lambda_6[0][0]                   
__________________________________________________________________________________________________
conv1d_14 (Conv1D)              (None, 256, 16)      272         spatial_dropout1d_6[0][0]        
__________________________________________________________________________________________________
add_6 (Add)                     (None, 256, 16)      0           add_5[0][0]                      
                                                                 conv1d_14[0][0]                  
__________________________________________________________________________________________________
conv1d_15 (Conv1D)              (None, 256, 16)      4112        add_6[0][0]                      
__________________________________________________________________________________________________
activation_7 (Activation)       (None, 256, 16)      0           conv1d_15[0][0]                  
__________________________________________________________________________________________________
lambda_7 (Lambda)               (None, 256, 16)      0           activation_7[0][0]               
__________________________________________________________________________________________________
spatial_dropout1d_7 (SpatialDro (None, 256, 16)      0           lambda_7[0][0]                   
__________________________________________________________________________________________________
conv1d_16 (Conv1D)              (None, 256, 16)      272         spatial_dropout1d_7[0][0]        
__________________________________________________________________________________________________
add_7 (Add)                     (None, 256, 16)      0           add_6[0][0]                      
                                                                 conv1d_16[0][0]                  
__________________________________________________________________________________________________
conv1d_17 (Conv1D)              (None, 256, 16)      4112        add_7[0][0]                      
__________________________________________________________________________________________________
activation_8 (Activation)       (None, 256, 16)      0           conv1d_17[0][0]                  
__________________________________________________________________________________________________
lambda_8 (Lambda)               (None, 256, 16)      0           activation_8[0][0]               
__________________________________________________________________________________________________
spatial_dropout1d_8 (SpatialDro (None, 256, 16)      0           lambda_8[0][0]                   
__________________________________________________________________________________________________
conv1d_18 (Conv1D)              (None, 256, 16)      272         spatial_dropout1d_8[0][0]        
__________________________________________________________________________________________________
add_8 (Add)                     (None, 256, 16)      0           add_7[0][0]                      
                                                                 conv1d_18[0][0]                  
__________________________________________________________________________________________________
conv1d_19 (Conv1D)              (None, 256, 16)      4112        add_8[0][0]                      
__________________________________________________________________________________________________
activation_9 (Activation)       (None, 256, 16)      0           conv1d_19[0][0]                  
__________________________________________________________________________________________________
lambda_9 (Lambda)               (None, 256, 16)      0           activation_9[0][0]               
__________________________________________________________________________________________________
spatial_dropout1d_9 (SpatialDro (None, 256, 16)      0           lambda_9[0][0]                   
__________________________________________________________________________________________________
conv1d_20 (Conv1D)              (None, 256, 16)      272         spatial_dropout1d_9[0][0]        
__________________________________________________________________________________________________
add_9 (Add)                     (None, 256, 16)      0           add_8[0][0]                      
                                                                 conv1d_20[0][0]                  
__________________________________________________________________________________________________
conv1d_21 (Conv1D)              (None, 256, 16)      4112        add_9[0][0]                      
__________________________________________________________________________________________________
activation_10 (Activation)      (None, 256, 16)      0           conv1d_21[0][0]                  
__________________________________________________________________________________________________
lambda_10 (Lambda)              (None, 256, 16)      0           activation_10[0][0]              
__________________________________________________________________________________________________
spatial_dropout1d_10 (SpatialDr (None, 256, 16)      0           lambda_10[0][0]                  
__________________________________________________________________________________________________
conv1d_22 (Conv1D)              (None, 256, 16)      272         spatial_dropout1d_10[0][0]       
__________________________________________________________________________________________________
add_10 (Add)                    (None, 256, 16)      0           add_9[0][0]                      
                                                                 conv1d_22[0][0]                  
__________________________________________________________________________________________________
conv1d_23 (Conv1D)              (None, 256, 16)      4112        add_10[0][0]                     
__________________________________________________________________________________________________
activation_11 (Activation)      (None, 256, 16)      0           conv1d_23[0][0]                  
__________________________________________________________________________________________________
lambda_11 (Lambda)              (None, 256, 16)      0           activation_11[0][0]              
__________________________________________________________________________________________________
spatial_dropout1d_11 (SpatialDr (None, 256, 16)      0           lambda_11[0][0]                  
__________________________________________________________________________________________________
conv1d_24 (Conv1D)              (None, 256, 16)      272         spatial_dropout1d_11[0][0]       
__________________________________________________________________________________________________
add_11 (Add)                    (None, 256, 16)      0           add_10[0][0]                     
                                                                 conv1d_24[0][0]                  
__________________________________________________________________________________________________
conv1d_25 (Conv1D)              (None, 256, 16)      4112        add_11[0][0]                     
__________________________________________________________________________________________________
activation_12 (Activation)      (None, 256, 16)      0           conv1d_25[0][0]                  
__________________________________________________________________________________________________
lambda_12 (Lambda)              (None, 256, 16)      0           activation_12[0][0]              
__________________________________________________________________________________________________
spatial_dropout1d_12 (SpatialDr (None, 256, 16)      0           lambda_12[0][0]                  
__________________________________________________________________________________________________
conv1d_26 (Conv1D)              (None, 256, 16)      272         spatial_dropout1d_12[0][0]       
__________________________________________________________________________________________________
add_12 (Add)                    (None, 256, 16)      0           add_11[0][0]                     
                                                                 conv1d_26[0][0]                  
__________________________________________________________________________________________________
conv1d_27 (Conv1D)              (None, 256, 16)      4112        add_12[0][0]                     
__________________________________________________________________________________________________
activation_13 (Activation)      (None, 256, 16)      0           conv1d_27[0][0]                  
__________________________________________________________________________________________________
lambda_13 (Lambda)              (None, 256, 16)      0           activation_13[0][0]              
__________________________________________________________________________________________________
spatial_dropout1d_13 (SpatialDr (None, 256, 16)      0           lambda_13[0][0]                  
__________________________________________________________________________________________________
conv1d_28 (Conv1D)              (None, 256, 16)      272         spatial_dropout1d_13[0][0]       
__________________________________________________________________________________________________
add_13 (Add)                    (None, 256, 16)      0           add_12[0][0]                     
                                                                 conv1d_28[0][0]                  
__________________________________________________________________________________________________
conv1d_29 (Conv1D)              (None, 256, 16)      4112        add_13[0][0]                     
__________________________________________________________________________________________________
activation_14 (Activation)      (None, 256, 16)      0           conv1d_29[0][0]                  
__________________________________________________________________________________________________
lambda_14 (Lambda)              (None, 256, 16)      0           activation_14[0][0]              
__________________________________________________________________________________________________
spatial_dropout1d_14 (SpatialDr (None, 256, 16)      0           lambda_14[0][0]                  
__________________________________________________________________________________________________
conv1d_30 (Conv1D)              (None, 256, 16)      272         spatial_dropout1d_14[0][0]       
__________________________________________________________________________________________________
add_15 (Add)                    (None, 256, 16)      0           conv1d_2[0][0]                   
                                                                 conv1d_4[0][0]                   
                                                                 conv1d_6[0][0]                   
                                                                 conv1d_8[0][0]                   
                                                                 conv1d_10[0][0]                  
                                                                 conv1d_12[0][0]                  
                                                                 conv1d_14[0][0]                  
                                                                 conv1d_16[0][0]                  
                                                                 conv1d_18[0][0]                  
                                                                 conv1d_20[0][0]                  
                                                                 conv1d_22[0][0]                  
                                                                 conv1d_24[0][0]                  
                                                                 conv1d_26[0][0]                  
                                                                 conv1d_28[0][0]                  
                                                                 conv1d_30[0][0]                  
__________________________________________________________________________________________________
activation_15 (Activation)      (None, 256, 16)      0           add_15[0][0]                     
__________________________________________________________________________________________________
dense (Dense)                   (None, 256, 3)       51          activation_15[0][0]              
__________________________________________________________________________________________________
activation_16 (Activation)      (None, 256, 3)       0           dense[0][0]                      
==================================================================================================
Total params: 65,843
Trainable params: 65,843
Non-trainable params: 0
__________________________________________________________________________________________________
Epoch 1/4
 999/1000 [============================>.] - ETA: 0s - loss: 0.1099Epoch 1/4
 810/1000 [=======================>......] - ETA: 3s - loss: 0.1113
Epoch 00001: val_loss improved from inf to 0.11132, saving model to res/20201207_082948_model.h5
1000/1000 [==============================] - 159s 159ms/step - loss: 0.1100 - val_loss: 0.1113
Epoch 2/4
 999/1000 [============================>.] - ETA: 0s - loss: 0.0830Epoch 1/4
 811/1000 [=======================>......] - ETA: 2s - loss: 0.1283
Epoch 00002: val_loss did not improve from 0.11132
1000/1000 [==============================] - 154s 154ms/step - loss: 0.0831 - val_loss: 0.1285
Epoch 3/4
 999/1000 [============================>.] - ETA: 0s - loss: 0.0827Epoch 1/4
 811/1000 [=======================>......] - ETA: 3s - loss: 0.1090
Epoch 00003: val_loss improved from 0.11132 to 0.10919, saving model to res/20201207_082948_model.h5
1000/1000 [==============================] - 159s 159ms/step - loss: 0.0827 - val_loss: 0.1092
Epoch 4/4
 999/1000 [============================>.] - ETA: 0s - loss: 0.0814Epoch 1/4
 812/1000 [=======================>......] - ETA: 3s - loss: 0.1116
Epoch 00004: val_loss did not improve from 0.10919
1000/1000 [==============================] - 166s 166ms/step - loss: 0.0814 - val_loss: 0.1116
INFO:root:re-loading last best model
INFO:root:predicting
777/777 [==============================] - 14s 18ms/step
INFO:root:evaluating
INFO:root:[[3555131    6458   30969]
 [  12403   31631     274]
 [ 113317      55  228002]]
INFO:root:              precision    recall  f1-score   support

       noise      0.966     0.990     0.978   3592558
       pulse      0.829     0.714     0.767     44308
        sine      0.879     0.668     0.759    341374

    accuracy                          0.959   3978240
   macro avg      0.892     0.790     0.835   3978240
weighted avg      0.957     0.959     0.956   3978240

INFO:root:saving to res/20201207_082948_results.h5.
```

## Training using command-line scripts
The training function `das.train.train` and all its arguments are also accessible from the command line via `das train` for use on the terminal. See [here](/technical/cli) for a description of all command-line arguments. The command-line interface is generated with [defopt](https://defopt.readthedocs.io/en/stable/index.html).

For instance, training command above can be invoked from the command line:
```shell
das train --data-dir dat/dmel_single_raw.npy --save-dir res --model-name tcn --kernel-size 16 --nb-filters 16 --nb-hist 512 --nb-epoch 20 -i
```

Shell scripts are particularly useful if you want to fit a set of networks with with different configurations to optimize architecture. For instance, this scripts will fit networks with different numbers of TCN blocks (`nb_conv`) and filters (`nb_filters`):
```shell
#!/bin/bash
conda activate das

YSUFFIX="pulse"
MODELNAME='tcn'
DATADIR='../dat/dmel_single.npy'
SAVEDIR="res"

NB_HIST=2048
KERNEL_SIZE=32
NB_FILTERS=32
NB_CONV=3

for NB_CONV in 2 3 4
do
    for NB_FILTERS in 16 32 48 64
    do
        das train -i --nb-filters $NB_FILTERS --kernel-size $KERNEL_SIZE --nb-conv $NB_CONV --nb-hist $NB_HIST --save-dir $SAVEDIR --y-suffix $YSUFFIX --data-dir $DATADIR --model-name $MODELNAME
    done
done
```

A description of all command line arguments can be obtained by typing `das train --help` in a terminal:

In [3]:
!das train --help

usage: das train [-h] -d DATA_DIR [-y Y_SUFFIX] [--save-dir SAVE_DIR]
                 [--save-prefix SAVE_PREFIX] [-m MODEL_NAME]
                 [--nb-filters NB_FILTERS] [-k KERNEL_SIZE]
                 [--nb-conv NB_CONV] [-u [USE_SEPARABLE [USE_SEPARABLE ...]]]
                 [--nb-hist NB_HIST] [-i] [--no-ignore-boundaries]
                 [--batch-norm] [--no-batch-norm] [--nb-pre-conv NB_PRE_CONV]
                 [--pre-kernel-size PRE_KERNEL_SIZE]
                 [--pre-nb-filters PRE_NB_FILTERS] [--pre-nb-conv PRE_NB_CONV]
                 [-v VERBOSE] [--batch-size BATCH_SIZE] [--nb-epoch NB_EPOCH]
                 [--learning-rate LEARNING_RATE] [--reduce-lr]
                 [--no-reduce-lr] [--reduce-lr-patience REDUCE_LR_PATIENCE]
                 [-f FRACTION_DATA] [--seed SEED] [--batch-level-subsampling]
                 [--no-batch-level-subsampling] [-t] [--no-tensorboard]
                 [--log-messages] [--no-log-messages] [--nb-stacks NB_STACKS]
         